In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from IPython.display import display, HTML
import pandas as pd
from meteostat import Point, Daily
from datetime import datetime
import time

In [ ]:
shapefile_path = "data/National_USFS_Fire_Occurrence_Point_FL/National_USFS_Fire_Occurrence_Point_(Feature_Layer).shp"
gdf = gpd.read_file(shapefile_path)

print(f"Loaded {len(gdf)} fire records")
print(f"CRS: {gdf.crs}")

# First five rows of the GeoDataFrame
print(gdf.head())

# Get general info about the columns and data types
print(gdf.info())

# Check basic statistics of numerical columns
print(gdf.describe())


In [ ]:
#display(gdf.info())
#display(gdf.describe(include='all'))
gdf = gdf[(gdf['FIREYEAR'] >= 1980) & (gdf['FIREYEAR'] <= 2020)]
display(gdf.isnull().sum())
print(len(gdf))

In [ ]:
gdf = gdf[(gdf['FIREYEAR'] >= 1900) & (gdf['FIREYEAR'] <= 2024)]

plt.figure(figsize=(10, 5))
sns.histplot(gdf['FIREYEAR'].dropna(), bins=50)
plt.title('Fires Per Year')
plt.xlabel('Year')
plt.ylabel('Number of Fires')
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
sns.histplot(gdf['TOTALACRES'], bins=50, log_scale=(False, True))
plt.title('Distribution of Fire Sizes (Log Scale)')
plt.xlabel('Fire Size (Acres)')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
sns.countplot(data=gdf, x='SIZECLASS', order=sorted(gdf['SIZECLASS'].dropna().unique()))
plt.title('Fire Count by Size Class (A–G)')
plt.xlabel('Size Class')
plt.ylabel('Number of Fires')
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
# top 20 causes only
cause_counts = gdf['STATCAUSE'].value_counts().head(8)
sns.barplot(x=cause_counts.index, y=cause_counts.values)
plt.xticks(rotation=45)
plt.title('Fire Causes')
plt.xlabel('Cause')
plt.ylabel('Number of Fires')
plt.grid(True)
plt.show()

In [ ]:
gdf.plot(markersize=1, color='orangered', alpha=0.4, figsize=(12, 8))
plt.title('USFS Fire Occurrence Locations')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.xlim([-155.0, -50])
plt.ylim([20, 70]) 
plt.show()

In [ ]:
# sample the data - map is too slow with all data points
gdf_sample = gdf.sample(min(50000, len(gdf)))
center = [gdf.geometry.y.mean(), gdf.geometry.x.mean()]
m = folium.Map(location=center, zoom_start=5)

for _, row in gdf_sample.iterrows():
    if row.geometry is not None:  # Check if geometry exists
        folium.CircleMarker(
            location=[row.geometry.y, row.geometry.x],
            radius=1,
            color='red',
            fill=True,
            fill_opacity=0.3
        ).add_to(m)
    
m
# m.save("fire_occurrence_map.html")
# display(HTML('<a href="fire_occurrence_map.html" target="_blank">Open Interactive Map</a>'))

In [ ]:
# Re-load the USFS Fire Occurrence shapefile
gdf = gpd.read_file(shapefile_path)

# Ensure the FIREYEAR column is in integer format and filter for years 1980-2020
gdf['FIREYEAR'] = pd.to_numeric(gdf['FIREYEAR'], errors='coerce')
gdf = gdf[(gdf['FIREYEAR'] >= 1980) & (gdf['FIREYEAR'] <= 2020)]

# Clean any rows with missing values in LATITUDE, LONGITUDE, or FIRE_DATE
gdf = gdf.dropna(subset=['LATDD83', 'LONGDD83', 'DISCOVERYD'])

# Verify the first few rows
gdf[['OBJECTID', 'FIREYEAR', 'LATDD83', 'LONGDD83', 'DISCOVERYD']].head()

In [ ]:
# write the cleaned data out
# output_shapefile_path = "data/National_USFS_Fire_Occurrence_Point_FL/usfs_fire_occurrence_cleaned_1980_2020.shp"
# gdf.to_file(output_shapefile_path)

In [ ]:
def get_weather_data(lat, lon, date):
    date = datetime.strptime(date, '%Y-%m-%d')
    location = Point(lat, lon)
    data = Daily(location, start=date, end=date)
    weather_data = data.fetch()

    # print(weather_data.head())

    if not weather_data.empty:
        # Extract relevant weather data
        temp = weather_data['tavg'].values[0]
        prcp = weather_data['prcp'].values[0]
        wind_speed = weather_data['wspd'].values[0]
        pressure = weather_data['pres'].values[0]
        return temp, prcp, wind_speed, pressure
    else:
        return None, None, None, None  # In case of no data

In [ ]:
def enrich_fire_data_with_weather(gdf):
    rows_written = 0
    rows_read = 0
    with open('data/weather.csv', 'w') as f:
        f.write("OBJECTID,TEMPERATURE_AVERAGE,PRECIPITATION,WIND_SPEED,PRESSURE\n")
        # Loop through each fire record
        for index, row in gdf.iterrows():
            rows_read += 1
            lat = row['LATDD83']
            lon = row['LONGDD83']
            fire_date = pd.to_datetime(row['DISCOVERYD'], errors='coerce')        
            
            fire_date_str = fire_date.strftime('%Y-%m-%d')
            
            # Get weather data for this fire's date and location
            temp, prcp, wind_speed, pressure = get_weather_data(lat, lon, fire_date_str)
            if temp and prcp and wind_speed and pressure:
                rows_written += 1
                row_output = f"{row['OBJECTID']},{temp},{prcp},{wind_speed},{pressure}\n"
                f.write(row_output)\

            if rows_read % 100 == 0:
                print(f"Index {rows_read} - Wrote {rows_written} rows")
            
            # Sleep for a short time to avoid hitting the API rate limits
            #time.sleep(0.1)  # Adjust based on rate limits (e.g., 60 requests per minute)


In [ ]:
# Load data from 1980-2020
# shapefile_path = "data/National_USFS_Fire_Occurrence_Point_FL/usfs_fire_occurrence_cleaned_1980_2020.shp"
# gdf = gpd.read_file(shapefile_path)

#print(f"Total rows to process: {len(gdf)}")

# Enrich the fire data with weather information and save it to CSV
# enrich_fire_data_with_weather(gdf)


In [ ]:
# with open('data/weather_per_fire.csv', newline='') as infile, open('data/weather_data_per_fire.csv', 'w', newline='') as outfile:
#    for line in infile:
#        outfile.write(line.replace('.', ',', 1))

In [ ]:
import pandas as pd
import geopandas as gpd

# Load data
weather = pd.read_csv('data/weather_data_per_fire.csv')
elevation = pd.read_csv('data/elevations_per_fire.csv')
usfs = gpd.read_file('data/National_USFS_Fire_Occurrence_Point_FL/usfs_fire_occurrence_cleaned_1980_2020.shp')

usfs['LATITUDE'] = usfs.geometry.y
usfs['LONGITUDE'] = usfs.geometry.x

# Merge on OBJECTID
merged = weather.merge(elevation, on='OBJECTID').merge(usfs, on='OBJECTID')

# Select required columns
#output = merged[['OBJECTID', 'TEMPERATURE_AVERAGE', 'PRECIPITATION', 'PRESSURE', 'ELEVATION',
#                 'FIRENAME', 'FIREYEAR', 'DISCOVERYD', 'FIREOUTDAT', 'TOTALACRES',
#                 'LATITUDE', 'LONGITUDE']]
len(merged)
# Save to CSV
output.to_csv('merged_fire_weather_elevation_data.csv', index=False)

In [ ]:
print(len(merged))
print(len(usfs))
print(len(elevation))
print(len(weather))
weather['OBJECTID'] = weather['OBJECTID'].astype(str)
elevation['OBJECTID'] = elevation['OBJECTID'].astype(str)
usfs['OBJECTID'] = usfs['OBJECTID'].astype(str)
weather['OBJECTID'] = weather['OBJECTID'].str.strip()
elevation['OBJECTID'] = elevation['OBJECTID'].str.strip()
merge1 = weather.merge(usfs, on='OBJECTID')
print(len(merge1))

merge2 = merge1.merge(elevation, on='OBJECTID')
print(len(merge2))

In [ ]:
m = weather.merge(elevation, on='OBJECTID')

In [ ]:
weather = weather.dropna(subset=['OBJECTID'])
elevation = elevation.dropna(subset=['OBJECTID'])
weather_duplicates = weather[weather.duplicated(subset=['OBJECTID'], keep=False)]
elevation_duplicates = elevation[elevation.duplicated(subset=['OBJECTID'], keep=False)]
print(len(weather))
print(len(elevation))
print(weather.columns)
print(elevation.columns)

In [ ]:
import csv

# Read the first CSV file into a dictionary with the first column as the key
file1_dict = {}
with open('data/weather_data_per_fire.csv', mode='r') as file1:
    reader = csv.reader(file1)
    for row in reader:
        file1_dict[row[0]] = row[1:]

# Read the second CSV file and merge it with the first dictionary
merged_dict = {}
with open('data/elevations_per_fire.csv', mode='r') as file2:
    reader = csv.reader(file2)
    for row in reader:
        if row[0] in file1_dict:
            merged_dict[row[0]] = file1_dict[row[0]] + row[1:]

# Print or use the merged dictionary
print(merged_dict)

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx
from shapely.geometry import box

# Load fire occurrence shapefile
usfs = gpd.read_file("data/National_USFS_Fire_Occurrence_Point_FL/usfs_fire_occurrence_cleaned_1980_2020.shp")

# Define the bounding box for the contiguous US + Alaska (in latitude/longitude)
bounding_box = [-125, 24, -66.9, 49.384358]  # USA bounding box (including Alaska)
bbox = box(*bounding_box)  # Create a bounding box geometry

# Filter the fire occurrence data to include only points within the bounding box
usfs = usfs[usfs.geometry.within(bbox)]

# Project to Web Mercator (EPSG:3857) for correct mapping
usfs = usfs.to_crs(epsg=3857)

# Extract X and Y coordinates
x = usfs.geometry.x
y = usfs.geometry.y

# Plotting
fig, ax = plt.subplots(figsize=(12, 8))

# Create a scatter plot (heatmap style) based on the fire occurrence coordinates
heatmap = ax.scatter(x, y, c='r', alpha=0.5, s=5)  # Red color with some transparency


# Customize plot
ax.set_title('USFS Fire Occurrences (1980–2020) - Heatmap')
ax.set_axis_off()  # Remove the axis for a cleaner look

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx
from shapely.geometry import box

# Load fire occurrence shapefile
usfs = gpd.read_file("data/National_USFS_Fire_Occurrence_Point_FL/usfs_fire_occurrence_cleaned_1980_2020.shp")

# Define the bounding box for the contiguous US + Alaska
# Coordinates: (xmin, ymin, xmax, ymax) in latitude/longitude
bounding_box = [-125, 24, -66.9, 49.384358]  # USA bounding box (including Alaska)
bbox = box(*bounding_box)  # Create a bounding box geometry

# Filter the data based on the bounding box
usfs = usfs[usfs.geometry.within(bbox)]

# Project the data to Web Mercator (meters) for accurate area and distance representation
usfs = usfs.to_crs(epsg=3857)

# Extract X and Y for hexbin
x = usfs.geometry.x
y = usfs.geometry.y

# Plotting
fig, ax = plt.subplots(figsize=(12, 8))

# Create a hexbin plot
hb = ax.hexbin(x, y, gridsize=100, cmap='hot', mincnt=1)

# Add basemap (Contextily) for context
ctx.add_basemap(ax, crs=usfs.crs, source=ctx.providers.Stamen.TonerLite)

# Customize plot
cb = fig.colorbar(hb, ax=ax)
cb.set_label('Number of Fires')
ax.set_title('USFS Fire Occurrences (1980–2020) - Hexbin Heatmap')

# Remove axis for cleaner presentation
ax.set_axis_off()

# Show the plot
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import geopandas as gpd

# Assuming your GeoDataFrame is loaded in 'gdf'
# Group the data by year and calculate the average for each weather parameter
grouped = merged.groupby('FIREYEAR').agg({
    'TEMPERATURE_AVERAGE': 'mean',
    'WIND_SPEED': 'mean',
    'PRESSURE': 'mean',
    'PRECIPITATION': 'mean'
}).reset_index()

# Create a 2x2 grid for plotting
fig, axs = plt.subplots(2, 2, figsize=(12, 10))

# Plot 1: Average Temperature
axs[0, 0].plot(grouped['FIREYEAR'], grouped['TEMPERATURE_AVERAGE'], color='tab:red', marker='o')
axs[0, 0].set_title('Average Temperature (°C)')
axs[0, 0].set_xlabel('Year')
axs[0, 0].set_ylabel('Avg Temperature (°C)')
axs[0, 0].grid(True)

# Plot 2: Average Wind Speed
axs[0, 1].plot(grouped['FIREYEAR'], grouped['WIND_SPEED'], color='tab:blue', marker='o')
axs[0, 1].set_title('Average Wind Speed (m/s)')
axs[0, 1].set_xlabel('Year')
axs[0, 1].set_ylabel('Avg Wind Speed (m/s)')
axs[0, 1].grid(True)

# Plot 3: Average Pressure
axs[1, 0].plot(grouped['FIREYEAR'], grouped['PRESSURE'], color='tab:green', marker='o')
axs[1, 0].set_title('Average Pressure (hPa)')
axs[1, 0].set_xlabel('Year')
axs[1, 0].set_ylabel('Avg Pressure (hPa)')
axs[1, 0].grid(True)

# Plot 4: Average Precipitation
axs[1, 1].plot(grouped['FIREYEAR'], grouped['PRECIPITATION'], color='tab:orange', marker='o')
axs[1, 1].set_title('Average Precipitation (mm)')
axs[1, 1].set_xlabel('Year')
axs[1, 1].set_ylabel('Avg Precipitation (mm)')
axs[1, 1].grid(True)

# Adjust layout for better spacing
plt.tight_layout()

# Show the plots
plt.show()


In [ ]:
import matplotlib.pyplot as plt

world = gpd.read_file("data/vectors/ne_110m_admin_0_countries.shp") 
usa = world[world['NAME'] == 'United States of America']
plt.figure(figsize=(10, 8))
# Scatter plot with longitude/latitude and elevation as the color
plt.scatter(gdf.geometry.x, gdf.geometry.y, c=gdf['ELEVATION'], cmap='terrain', s=20, alpha=0.6)
# Add the USA outline to the plot
usa.boundary.plot(ax=plt.gca(), linewidth=2, color='black')
plt.colorbar(label='Elevation (meters)')
plt.title('Fire Locations with Elevation')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
# set scale for usa only
plt.xlim([-180, -65]) 
plt.ylim([20, 85])

# Show the plot
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

filtered = merged
bin_edges = np.arange(0, 4250, 250)  # From 0 to 4000, step 250
filtered['elevation_bin'] = pd.cut(filtered['ELEVATION'], bins=bin_edges)
elevation_counts = filtered['elevation_bin'].value_counts().sort_index()

plt.figure(figsize=(12, 6))
sns.barplot(x=elevation_counts.index.astype(str), y=elevation_counts.values, palette="terrain")
plt.xticks(rotation=45, ha='right')
plt.xlabel('Elevation Range (m)')
plt.ylabel('Number of Fires')
plt.title('Wildfire Occurrences by Elevation')
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = merged
df['FIREOUTDAT'] = df['FIREOUTDAT'].replace('0207/11/08', '2017/11/07')
df['FIREOUTDAT'] = df['FIREOUTDAT'].replace('0216/08/12', '2016/08/12')


# Optional: Convert duration if it's not already calculated
if 'FIRE_DURATION' not in df.columns and {'DISCOVERYD', 'FIREOUTDAT'}.issubset(df.columns):
    df['FIRE_DURATION'] = (pd.to_datetime(df['FIREOUTDAT']) - pd.to_datetime(df['DISCOVERYD'])).dt.days

# Keep only relevant numeric columns
corr_columns = [
    'TOTALACRES', 'FIRE_DURATION', 'TEMPERATURE_AVERAGE',
    'PRECIPITATION', 'WIND_SPEED', 'PRESSURE', 'ELEVATION'
]
df_corr = df[corr_columns].dropna()

# Compute correlation matrix
corr_matrix = df_corr.corr()

print(corr_matrix)

# Plot correlation heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, fmt=".2f")
plt.title('Correlation Matrix of Fire & Environmental Variables')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=45, ha='right')
plt.show()